In [1]:
from pathlib import Path
import numpy as np
import pprint
import json
import torch
import gc
import traceback
import time
from scipy.io import savemat
from datetime import datetime
from myTools.output_converter import convert_to_matlab_files_ks4
from kilosort.run_kilosort import load_sorting
from probeinterface.io import write_probeinterface

from myTools.read_simulation import get_simulation_path
from myTools.set_gain_and_offset import set_gain_and_offset
from myTools.init_run import get_recording, get_probe, get_probe_sorted, set_probe_info, get_stimtime

import spikeinterface.full as si
# import spikeinterface.extractors as se
import spikeinterface.preprocessing as spre
import spikeinterface.sorters as ss
# import spikeinterface.postprocessing as spost
# import spikeinterface.qualitymetrics as sqm
# import spikeinterface.comparison as sc
import spikeinterface.exporters as sexp
# import spikeinterface.curation as scur
# import spikeinterface.widgets as sw

from kilosort.plots import plot_drift_amount, plot_drift_scatter, plot_diagnostics, plot_spike_positions
from cellactivityrecodingsimulator import Recording

In [9]:
### Select experiment ###
dir_info = {
    "root_dir": r"C:\Users\tanaka-users\tlab\yasui\simulations",
    "ep": "001",
    "cd": "001",
}
dict_path = get_simulation_path(dir_info)

recording = si.load(dict_path["recording"])
probe = recording.get_probe()

### Setting sorters ###
do_preprocess = False
do_runsort = True
do_export_phy = False
sorter = "kilosort4"

sort_params = ss.get_default_sorter_params("kilosort4")
# Kilosort4用パラメータ
name_thisparam = "default"
sort_params["torch_device"] = "cuda"
sort_params["save_extra_vars"] = True
sort_params["skip_kilosort_preprocessing"] = do_preprocess

In [3]:
### Preprocess recording ###
if do_preprocess:

    print("\n" + "="*20 + " parameter " + "="*20)
    print(f"\n{'='*5} {sorter} {'='*5}")
    pprint.pprint(sort_params)

    params_file = dict_path["exp"] / sorter / name_thisparam / "params.txt"
    params_file.parent.mkdir(parents=True, exist_ok=True)
    with open(str(params_file), "w", encoding="utf-8") as f:
        json.dump(sort_params, f, indent=4, ensure_ascii=False)

    print("="*5, sorter, "="*5)
    folder = dict_path["exp"] / sorter / name_thisparam
    pp_rec_folder = folder / "pp_rec"

    recording_f = spre.bandpass_filter(recording, freq_min=300, freq_max=3000)
    recording_cmr = spre.common_reference(recording_f, reference="global")
    recording_whiten = spre.whiten(recording_cmr, int_scale=200)
    recording_motion = spre.correct_motion(recording_whiten, preset="kilosort_like")
    recording_preprocessed = recording_motion.save(format="binary", folder=pp_rec_folder, overwrite=True)
    recording = recording_preprocessed
else:
    print("skip preprocessing.")


skip preprocessing.


In [3]:
if do_runsort:
    print(f"{'='*5} {sorter} {'='*5}")
    print(f"  {sorter}実行開始: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    try:
        start_time = time.time()
        
        sorter_output_dir = dict_path["exp"] / sorter / name_thisparam / "sorting"
        sorting = ss.run_sorter(
            sorter_name=sorter,
            folder=sorter_output_dir, 
            remove_existing_folder=True, 
            recording=recording,
            verbose=True,
            **sort_params
            )
            
        elapsed_time = time.time() - start_time
        print(f"  {sorter}実行完了: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} (経過時間: {elapsed_time/60:.1f}分)")

        print(f"  Analyzer作成開始: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        analyzer = si.create_sorting_analyzer(sorting=sorting, recording=recording, format='binary_folder', folder=dict_path["exp"] / sorter / name_thisparam / "analyzer", overwrite=True)
        print(analyzer)
        print("===== Sorting done =====")

    except Exception as e:
        print(f"Error occurred while running {sorter}: {e}")
        traceback.print_exc()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            gc.collect()
            print(f"  GPUメモリクリア完了 - 割り当て済み: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")
else:
    print("skip sorting.")


kilosort.run_kilosort:  
kilosort.run_kilosort: Computing preprocessing variables.
kilosort.run_kilosort: ----------------------------------------


===== kilosort4 =====
  kilosort4実行開始: 2026-02-13 14:04:01


kilosort.run_kilosort: N samples: 300000
kilosort.run_kilosort: N seconds: 10.0
kilosort.run_kilosort: N batches: 5
kilosort.run_kilosort: Preprocessing filters computed in 14.74s; total 14.75s
kilosort.run_kilosort:  
kilosort.run_kilosort: Resource usage after preprocessing
kilosort.run_kilosort: ********************************************************
kilosort.run_kilosort: CPU usage:    13.00 %
kilosort.run_kilosort: Mem used:     89.30 %     |      28.04 GB
kilosort.run_kilosort: Mem avail:     3.36 / 31.40 GB
kilosort.run_kilosort: ------------------------------------------------------
kilosort.run_kilosort: GPU usage:    `conda install pynvml` for GPU usage
kilosort.run_kilosort: GPU memory:   21.77 %     |      5.20   /    23.89 GB
kilosort.run_kilosort: Allocated:     0.04 %     |      0.01   /    23.89 GB
kilosort.run_kilosort: Max alloc:     5.11 %     |      1.22   /    23.89 GB
kilosort.run_kilosort: ********************************************************
kilosort.run_kil

kilosort4 run time 38.46s
  kilosort4実行完了: 2026-02-13 14:04:39 (経過時間: 0.6分)
  Analyzer作成開始: 2026-02-13 14:04:39


estimate_sparsity (no parallelization):   0%|          | 0/10 [00:00<?, ?it/s]

SortingAnalyzer: 384 channels - 20 units - 1 segments - binary_folder - sparse - has recording
Loaded 0 extensions
===== Sorting done =====


c:\Users\tanaka-users\tlab\yasui\mySpikeinterface\.venv\Lib\site-packages\spikeinterface\core\basesorting.py:386: UserWarning: The registered recording will not be persistent on disk, but only available in memory
  warnings.warn("The registered recording will not be persistent on disk, but only available in memory")


In [ ]:
### export to phy ###
if do_export_phy:
    analyzer = si.load_sorting_analyzer(folder= dict_path["exp"] / sorter / name_thisparam / "analyzer")
    analyzer.compute(['random_spikes', 'waveforms', 'templates'])
    # PC特徴の計算が時間かかってそうなので、Falseにしておく
    sexp.export_to_phy(sorting_analyzer=analyzer, output_folder=dict_path["exp"] / sorter / name_thisparam / "phy", remove_if_exists=True,
                            compute_pc_features=False)
else:
    print("skip export to phy.")


compute_waveforms (no parallelization):   0%|          | 0/10 [00:00<?, ?it/s]

write_binary_recording (no parallelization):   0%|          | 0/10 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [14]:
### export to matlab files for ks4 ###
convert_to_matlab_files_ks4(res_dir=dict_path["exp"] / sorter / name_thisparam, recording=recording)

### export to probe.json ###
write_probeinterface(dict_path["exp"] / sorter / "probe.json", probe_or_probegroup=probe)

### plot drift amount, scatter, diagnostics, spike positions ###
ks_dir = dict_path["exp"] / sorter / name_thisparam / "sorting" / "sorter_output"
ops, st, clu, similar_templates, \
    is_ref, est_contam_rate, kept_spikes, \
        tF, Wall, full_st, full_clu, full_amp = \
            load_sorting(ks_dir, device="cuda", load_extra_vars=True)

plot_drift_amount(ops, ks_dir)
plot_drift_scatter(full_st, ks_dir)
plot_diagnostics(Wall, full_clu, ops, ks_dir)
plot_spike_positions(clu, is_ref, ks_dir)


# stim_times_rise, stim_times_fall, fs_obx = get_stimtime(meta_obx, dict_path["obx"]["bin"])
# savemat(dict_path["exp"] / sorter / name_thisparam / "stim_times.mat", {'stim_times_rise': stim_times_rise, 'stim_times_fall': stim_times_fall, 'fs': fs_obx})
